<a href="https://colab.research.google.com/github/alyatwa/2DConf/blob/master/OCRusingTesseract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:

!sudo apt-get install tesseract-ocr
!sudo apt-get install tesseract-ocr-ara
!sudo apt install libtesseract-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-ara
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 645 kB of archives.
After this operation, 1,447 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-ara all 1:4.00~git30-7274cfa-1.1 [645 kB]
Fetched 645 kB in 1s (743 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: R

In [41]:
!pip install pytesseract
!pip install --upgrade pymupdf
!pip install pyarabic
!pip install ArabicOcr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 19.1 MB/s eta 0:00:00


In [24]:
ls -l

total 560
-rw-r--r-- 1 root root    310 Dec 12 07:01 extracted_arabic_text.txt
drwxr-xr-x 1 root root   4096 Dec  9 14:42 sample_data/
-rw-r--r-- 1 root root 563064 Dec 12 06:53 ل222.pdf


In [42]:

import shutil
import os
from ArabicOcr import arabicocr
import random
import pytesseract
import pyarabic.araby as araby
import pymupdf
import io
try:
    from PIL import Image
except ImportError:
    import Image

In [1]:
from google.colab import files

uploaded = files.upload()

Saving ل222.pdf to ل222.pdf


In [ ]:


def extract_arabic_text_advanced(pdf_path):
    """
    Extract Arabic text from PDF using multiple methods
    """
    pdf_document = pymupdf.open(pdf_path)
    full_text = ""

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)

        # METHOD 1: Try direct text extraction first (for text-based PDFs)

        text_direct_ = page.get_text()
        text_direct = araby.strip_tashkeel(text_direct_)

        if text_direct and len(text_direct.strip()) < 2:  # If we got reasonable text
            full_text += f"=================== Page {page_num + 1} (Direct Extraction) ==================\n"
            full_text += text_direct + "\n\n"
        else:
            # METHOD 2: Use OCR with higher quality settings
            # Increase resolution for better OCR
            zoom = 1  # Increase for better quality
            mat = pymupdf.Matrix(zoom, zoom)

            # Get image with higher resolution
            pix = page.get_pixmap(matrix=mat, dpi=300)
            pix.save("page-%i.png" % page.number)  # store image as a PNG
            img_data = pix.tobytes("png")

            # Convert to PIL Image
            image = Image.open(io.BytesIO(img_data))


            image_path='page-0.png'
            out_image='out.jpg'
            results=arabicocr.arabic_ocr(image_path,out_image)
            full_text += "===========*****************************************************============="
            print(results)

            # OCR Configuration for Arabic
            custom_config = r'--oem 3 --psm 6 -l ara'
            text_ = pytesseract.image_to_string(image, config=custom_config, lang='ara')

            text_ocr = araby.strip_tashkeel(text_)

            full_text += f"=== Page {page_num + 1} (OCR Extraction) ===\n"
            full_text += text_ocr + "\n\n"

            # Optional: Save OCR image for debugging
            # image.save(f"page_{page_num+1}_debug.png")

    pdf_document.close()
    return full_text

def extract_with_clean_layout(pdf_path):
    """
    Extract text with layout preservation using PyMuPDF's text extraction options
    """
    pdf_document = pymupdf.open(pdf_path)
    full_text = ""

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)

        # Extract text with different layout options
        # Option 1: Preserve layout
        text_ = page.get_text("text", sort=True)
        text = araby.strip_tashkeel(text_)

        # Option 2: HTML-like structure (but we'll convert to plain text)
        blocks = page.get_text("dict")["blocks"]

        page_text = f"===== Page {page_num + 1} =====\n"

        # Process text blocks
        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    for span in line["spans"]:
                        if "text" in span and span["text"].strip():
                            page_text += span["text"] + " "
                    page_text += "\n"
                page_text += "\n"

        full_text += page_text + "\n\n"

    pdf_document.close()
    return full_text

def extract_all_text_methods(pdf_path):
    """
    Try multiple extraction methods and combine results
    """
    print(f"Processing: {pdf_path}")
    print(f"File size: {os.path.getsize(pdf_path)} bytes")

    # Method 1: Direct extraction with layout
    print("\n--- Method 1: Direct text extraction --**************************************-")
    #text1 = extract_with_clean_layout(pdf_path)

    # Method 2: Advanced extraction
    print("--- Method 2: Advanced extraction --+++++++++++++++++++++++++-")
    text2 = extract_arabic_text_advanced(pdf_path)

    return text1, text2

def save_results(text1, text2, base_name="ل222"):
    """Save extraction results"""
    with open(f"{base_name}_method1.txt", "w", encoding="utf-8") as f:
        f.write(text1)

    with open(f"{base_name}_method2.txt", "w", encoding="utf-8") as f:
        f.write(text2)

    print(f"\nResults saved to:")
    print(f"1. {base_name}_method1.txt (Direct extraction)")
    print(f"2. {base_name}_method2.txt (Advanced extraction)")

if __name__ == "__main__":
    pdf_file = "ل222.pdf"

    if not os.path.exists(pdf_file):
        print(f"Error: File '{pdf_file}' not found!")
        exit(1)

    try:
        text1, text2 = extract_all_text_methods(pdf_file)

        print("\n" + "="*60)
        print("METHOD 1 RESULTS (Direct Extraction):")
        print("="*60)
        print(text1[:2000])  # Show first 2000 chars

        print("\n" + "="*60)
        print("METHOD 2 RESULTS (Advanced Extraction):")
        print("="*60)
        print(text2[:2000])  # Show first 2000 chars

        save_results(text1, text2)

    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

# Alternative: Simple extraction that might work better
print("\n" + "="*60)
print("SIMPLE EXTRACTION METHOD (Try this first):")
print("="*60)

def simple_extract(pdf_path):
    doc = pymupdf.open(pdf_path)
    all_text = ""

    for page_num in range(len(doc)):
        page = doc[page_num]

        # Get all text with different encoding
        text_ = page.get_text("text", sort=True)
        text = araby.strip_tashkeel(text_)

        # Also try raw extraction
        raw_text = page.get_text("rawdict")

        all_text += f"\n{'='*40}\nPage {page_num + 1}\n{'='*40}\n"
        all_text += text + "\n"

        # Try to extract from spans
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    line_text = " ".join([span.get("text", "") for span in line["spans"]])
                    if line_text.strip():
                        all_text += line_text + "\n"

    doc.close()
    return all_text

# Run simple extraction
try:
    simple_text = simple_extract(pdf_file)
    with open("ل222_simple.txt", "w", encoding="utf-8") as f:
        f.write(simple_text)
    print("Simple extraction saved to: ل222_simple.txt")
    print("First 1000 characters:")
    print(simple_text[:1000])
except Exception as e:
    print(f"Simple extraction failed: {e}")

Processing: ل222.pdf
File size: 563064 bytes

--- Method 1: Direct text extraction --**************************************-
--- Method 2: Advanced extraction --+++++++++++++++++++++++++-


[INFO] OCR'ing input image...


In [8]:
print(extractedInformation)

NameError: name 'extractedInformation' is not defined